In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils.utils import *
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

## Data preparation

### Load data

In [ ]:
file_path = r'/home/jovyan/descartes_tech_interview/Data/'
train = pd.read_csv(file_path + 'train_auto.csv')
test = pd.read_csv(file_path + 'test_auto.csv')

In [ ]:
train.columns

In [ ]:
# Only consider TARGET_FLAG
train = train.drop('TARGET_AMT', axis=1)
test = test.drop('TARGET_AMT', axis=1)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
train_y = train[['INDEX', 'TARGET_FLAG']]
train_x = train.loc[:, train.columns != 'TARGET_FLAG']

In [ ]:
test_y = test[['INDEX', 'TARGET_FLAG']]
test_x = test.loc[:, test.columns != 'TARGET_FLAG']

### Deal with Categorical columns

In [ ]:
train.dtypes

In [ ]:
numeric_col = []
categoric_col = []
for col in train_x.columns:
    if train[col].dtypes != object:
        numeric_col.append(col)
    else:
        categoric_col.append(col)

In [ ]:
categoric_col

In [ ]:
# Handle str columns: 2 types
true_categorical_cols = ['PARENT1', 'MSTATUS', 'SEX', 'EDUCATION', 'JOB',
                        'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED',
                        'URBANICITY']
num_categorical_cols = ['INCOME', 'HOME_VAL', 'BLUEBOOK',  'OLDCLAIM']

### Train data

In [ ]:
# Categorical columns: transfer into nominal numeric variables
# Store encoding labels into dict and csv file
train_true_cat, labels_dict = to_nominal_df(train_x, true_categorical_cols)
df_labels = pd.DataFrame(list(labels_dict.items()), columns=['col_name', 'values'])
df_labels.to_csv(file_path + 'numerical_labels.csv')

# Money values in str: transfer to float
train_num_cat = to_numerical_df(train_x, num_categorical_cols)

In [ ]:
# Merge treated categorical features
train_num = pd.merge(train_true_cat, train_num_cat, on=['INDEX'])

# Merge all
train_num = pd.merge(train_num, train_x[numeric_col], on=['INDEX'])

### Test data

In [ ]:
# Categorical columns: transfer into nominal numeric variables
test_true_cat = to_nominal_df_test(test_x, labels_dict)

# Money values in str: transfer to float
test_num_cat = to_numerical_df(test_x, num_categorical_cols)

In [ ]:
# Merge treated categorical features
test_num = pd.merge(test_true_cat, test_num_cat, on=['INDEX'])

# Merge all
test_num = pd.merge(test_num, test_x[numeric_col], on=['INDEX'])

In [ ]:
test_num.dtypes

### Handle missing values

In [ ]:
imputer = SimpleImputer(missing_values=np.nan,
                        strategy='median')

#### Train data

In [ ]:
train_num.isna().sum()

In [ ]:
for col in train_num.columns[test_num.isnull().any()]:
    train_num[col] = imputer.fit_transform(train_num[col].values.reshape(-1, 1))

In [ ]:
train_num.isna().sum()

#### Test data

In [ ]:
test_num.isna().sum()

In [ ]:
for col in test_num.columns[test_num.isnull().any()]:
    test_num[col] = imputer.fit_transform(test_num[col].values.reshape(-1, 1))

In [ ]:
test_num.isna().sum()

## Save file

In [ ]:
train_num['TARGET_FLAG'] = train['TARGET_FLAG']
train_num.to_csv(file_path + 'train_num.csv', index=False)

In [ ]:
test_num['TARGET_FLAG'] = test['TARGET_FLAG']
test_num.to_csv(file_path + 'test_num.csv', index=False)